In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import imagenet_utils
import numpy as np
import os
import json
import csv
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow # cv2.imshow does not work on Google Colab notebooks, --> use cv2_imshow instead

In [ ]:
# loading settings
model_string = "resnet50"  # inceptionv3, resnet50, vgg16
dataset_string = "imagenet"  # imagenet

# other settings
heatmap_intensity = 0.5
base_learning_rate = 0.0001

# data set
#path_to_dataset = '/content/gdrive/My Drive/What are CNNs looking at/files/New Masks Dataset/'
#path_to_dataset = '/content/gdrive/My Drive/Kaggle/files/Face Mask Dataset'
path_to_dataset = '/content/gdrive/.shortcut-targets-by-id/1wI0fw-edJDu1RgH2Ldk3n4VBJ4m4EJuv/combined'
other_path = "/content/gdrive/My Drive/What are CNNs looking at/New Masks Dataset/"

In [ ]:
def gradCAM(orig, model, model_string, DIM, HM_DIM, last_layer, classes, intensity=0.5, res=250):

    if model_string == "inceptionv3":
        from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions

    elif model_string == "resnet50":
        from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

    elif model_string == "vgg16":
        from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions

    else:  # use InceptionV3 and imagenet as default
        from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions


    img = image.load_img(orig, target_size=(DIM, DIM))

    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)
    for pred_it, label_it in zip(preds[0], classes):
      print(label_it, ": ", pred_it)
    print("preds")
    print(preds)
    prob = np.max(preds[0])
    index = list(preds[0]).index(prob)
    label = classes[index]
    label = "{}: {:.2f}%".format(label, prob * 100)
    print("[INFO] {}".format(label))

    with tf.GradientTape() as tape:
        last_conv_layer = model.get_layer(last_layer)
        iterate = tf.keras.models.Model([model.inputs], [model.output, last_conv_layer.output]) # run model and achive certain output 'last_conv_layer.output'
        model_out, last_conv_layer = iterate(x)
        class_out = model_out[:, np.argmax(model_out[0])]
        grads = tape.gradient(class_out, last_conv_layer) # class out: take derivative; last_conv_layer: variable to derive from
        pooled_grads = K.mean(grads, axis=(0, 1, 2))

    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, last_conv_layer), axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    heatmap = heatmap.reshape((HM_DIM, HM_DIM))

    img_original = cv2.imread(orig)
    heatmap = cv2.resize(heatmap, (img_original.shape[1], img_original.shape[0]))
    # plt.matshow(heatmap)
    # plt.show()

    heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    # plt.matshow(heatmap)
    # plt.show()
    img_heatmap = heatmap * intensity + img_original

    cv2_imshow(img_heatmap)

In [ ]:
# model dependent imports, weight loading and model dependent parameter setting
K.clear_session()
if model_string == "inceptionv3":
    from tensorflow.keras.applications.inception_v3 import InceptionV3
    from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
    Model = InceptionV3
    if dataset_string == "imagenet":
        model = InceptionV3(weights='imagenet')
    target_input_dimension = 299
    heatmap_dimension = 8
    last_layer_name = 'conv2d_93'
elif model_string == "resnet50":
    from tensorflow.keras.applications import ResNet50
    from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
    Model = ResNet50
    if dataset_string == "imagenet":
        model = ResNet50(weights="imagenet")
    target_input_dimension = 224
    heatmap_dimension = 7
    last_layer_name = 'conv5_block3_3_conv'
elif model_string == "vgg16":
    from tensorflow.keras.applications import VGG16
    from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
    Model = VGG16
    if dataset_string == "imagenet":
        model = VGG16(weights="imagenet")
    target_input_dimension = 224
    heatmap_dimension = 14
    last_layer_name = 'block5_conv3'
else:  # use InceptionV3 and imagenet as default
    from tensorflow.keras.applications.inception_v3 import InceptionV3
    from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
    if dataset_string == "imagenet":
        model = InceptionV3(weights='imagenet')
    target_input_dimension = 299
    heatmap_dimension = 8
    last_layer_name = 'conv2d_93'

102973440/102967424 [==============================] - 1s 0us/step


In [ ]:
# mount google drive to access database

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
class_names=['Mask', 'Non Mask']

In [ ]:
%cd /content/gdrive/MyDrive/
%ll

In [ ]:
%cd /content/gdrive/MyDrive
%cd /content/gdrive/.shortcut-targets-by-id/1CYXqSpprxFgIFpiRfRSoR_KL7ZIOXtOA/combined
#sos attempt path
path_to_dataset= '/content/gdrive/.shortcut-targets-by-id/1CYXqSpprxFgIFpiRfRSoR_KL7ZIOXtOA/combined/'

In [ ]:
# import database

train_dir = os.path.join(path_to_dataset, 'Train')
validation_dir = os.path.join(path_to_dataset, 'Validation')
test_dir = os.path.join(path_to_dataset, 'Test')
BATCH_SIZE = 64
IMG_SIZE = (224, 224)

train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)
validation_dataset = image_dataset_from_directory(validation_dir,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  image_size=IMG_SIZE)
test_dataset = image_dataset_from_directory(test_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

In [ ]:
# Sarra
#%cd /content/gdrive/.shortcut-targets-by-id/1D594PlXLC7S5YLnKD2s09vbezmGes8H6/What are CNNs looking at
#sos
%cd /content/gdrive/.shortcut-targets-by-id/1D594PlXLC7S5YLnKD2s09vbezmGes8H6/What are CNNs looking at
# Jan-Lukas
#%cd /content/gdrive/My Drive/What are CNNs looking at

model_new = tf.keras.models.load_model('saved_model/resnet50trained_wrongmask')
model_new.summary()

In [ ]:
%cd /content/gdrive/.shortcut-targets-by-id/1D594PlXLC7S5YLnKD2s09vbezmGes8H6/What are CNNs looking at
images = [ 'JLM_5.jpg', 'JLM_6.jpg', 'JLM_7.jpg', 'JLM_2a.jpg', 'JLM_3a.jpg', 'JLM_1a.jpg', 'maskWrong.PNG','nomask.PNG',
          'abin_halfmask.jpg', 'maskdown.PNG', 'maskdown2.PNG','maskdown3.PNG','maskchin.PNG','maskchin1.PNG']
DIM = 224
for orig in images:
  gradCAM(orig, model_new, model_string, target_input_dimension, heatmap_dimension,'tf.identity',class_names)


In [ ]:
%cd /content/gdrive/.shortcut-targets-by-id/1D594PlXLC7S5YLnKD2s09vbezmGes8H6/What are CNNs looking at
images = ['m'askWrong.PNG,'nomask.PNG','abin_halfmask.jpg', 'maskdown.PNG', 'maskdown2.PNG','maskdown3.PNG','maskchin.PNG','maskchin1.PNG']

DIM = 224
for orig in images:
  gradCAM(orig, model, model_string, target_input_dimension, heatmap_dimension,'tf.identity',class_names)